In [1]:
import numpy as np
import pandas as pd 
df=pd.read_csv('amazon_alexa.csv',sep='\t')
df.head(10)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [2]:
df.describe()

,rating,feedback
count,3150.000000,3150.000000
mean,4.463175,0.918413
std,1.068506,0.273778
min,1.000000,0.000000
25%,4.000000,1.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,1.000000


In [3]:
df.shape

(3150, 5)

In [4]:
df1=df[['verified_reviews','feedback']]

In [5]:
df1.columns=['review','label']

In [6]:
df1.head()

,review,label
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


In [7]:
df1.value_counts('label')

label
1    2893
0     257
Name: count, dtype: int64

In [8]:
#balancing the dataset for analysis (downsizing the data set )
label_counts=df1['label'].value_counts()
#counting the occurances of each label 
#get the number to drop from the majority class 
rows_drop=label_counts.max() - label_counts.min()

In [9]:
if rows_drop>0:
    data_majority=df1[df1["label"]==1]
    data_balanced=df1.drop(data_majority.sample(rows_drop).index)
else:
    data_balanced=df1.copy()
print(data_balanced['label'].value_counts())

label
1    257
0    257
Name: count, dtype: int64


In [10]:
import re

def clean_text(text):
  # Remove special characters and punctuation
  text = re.sub(r"[^\w\s]", " ", text)
  # Remove single characters
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)
  # Remove HTML tags
  text = re.sub(r"<[^>]*>", " ", text)
  # Lowercase the text
  text = text.lower()
  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text)
  # Trim leading and trailing spaces
  text = text.strip()

  return text

In [11]:
import pandas as pd
import re
# Assuming you have defined the clean_text function somewhere
# Extract the review column as a list
reviews = data_balanced['review'].tolist()
# Clean the text in the list
cleaned_reviews = []
for review in reviews:
    if isinstance(review, str):
        cleaned_reviews.append(clean_text(review))
    else:
        cleaned_reviews.append("")
# Add the cleaned reviews as a new column to the DataFrame
data_balanced['clean_reviews'] = cleaned_reviews

In [12]:
data_balanced.head(10)

,review,label,clean_reviews
9,Love it! I’ve listened to songs I haven’t hear...,1,love it ve listened to songs haven heard since...
15,"I love it, wife hates it.",1,love it wife hates it
35,I bought this to compare the speaker quality t...,1,bought this to compare the speaker quality to ...
37,Speaker is better than 1st generation Echo,1,speaker is better than 1st generation echo
46,"It's like Siri, in fact, Siri answers more acc...",0,it like siri in fact siri answers more accurat...
67,You’re all I need...na na nana!,1,you re all need na na nana
71,Works and sounds great.,1,works and sounds great
79,I love it. It does everything,1,love it it does everything
83,I purchased an echo dot for my husband for Fat...,1,purchased an echo dot for my husband for fathe...
84,Works fine!!,1,works fine


In [13]:
import pandas as pd
total_rows=len(data_balanced)
test_size=int(total_rows*0.95)
test_set=data_balanced.sample(test_size)
train_set=data_balanced.drop(test_set.index)

In [14]:
#Gemini API setting up
import pathlib
import textwrap
import os
import markdown
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text,'>',predicate=lambda _:True))

In [24]:
import os 
import key_scrt
genai.configure(api_key=key_scrt.GOOGLE_API_KEY)  # Assuming key_scrt contains GOOGLE_API_KEY

In [25]:
for m in genai.list_models():
    if 'generated content' in m.supported_generation_methods:
        print(m.name)

In [26]:
model=genai.GenerativeModel('gemini-pro')

In [27]:
response=model.generate_content("What is the meaning of physics ?")

to_markdown(response.text)

>**Physics** is a natural science that studies the fundamental constituents of the universe and the laws that govern their interactions. It is concerned with understanding the most basic building blocks of matter and energy, as well as the forces that act upon them.
>
>**Key Concepts of Physics:**
>
>* **Matter:** Anything that has mass and occupies space.
>* **Energy:** A conserved quantity that can take various forms, such as heat, light, or motion.
>* **Force:** An interaction that can change the motion of an object.
>* **Laws of Motion:** Describe the relationship between force, mass, and acceleration.
>* **Electromagnetism:** Study of the interactions between electric and magnetic fields.
>* **Thermodynamics:** Study of heat and its relation to other forms of energy.
>* **Quantum Mechanics:** Study of matter and energy at the atomic and subatomic level.
>* **Relativity:** Theory that describes the relationship between space, time, and gravity.
>
>**Branches of Physics:**
>
>* **Classical Physics:** Describes macroscopic phenomena using Newtonian mechanics, thermodynamics, and electromagnetism.
>* **Modern Physics:** Includes quantum mechanics, relativity theory, and particle physics, which deal with phenomena at atomic and subatomic scales.
>* **Applied Physics:** Uses physical principles to solve practical problems in engineering, medicine, and other fields.
>
>**Goals of Physics:**
>
>* To understand the fundamental laws that govern the universe.
>* To develop theories and models that describe natural phenomena.
>* To make predictions and test hypotheses based on these theories.
>* To advance technological applications based on our understanding of physics.

In [28]:
test_set_sample=test_set.sample(20)
test_set_sample['pred_label']=''
test_set_sample

,review,label,clean_reviews,pred_label
615,Took a little work to set up but I finally got...,1,took little work to set up but finally got it ...,
857,"Stopped working after 2 weeks ,didn't follow c...",0,stopped working after 2 weeks didn follow comm...,
1976,The Echo does not link to Direct tv even thoug...,0,the echo does not link to direct tv even thoug...,
374,,0,,
2639,Love the product but dislike that I can’t sync...,1,love the product but dislike that can sync up ...,
67,You’re all I need...na na nana!,1,you re all need na na nana,
672,Bought this to go in my niece's room. You can'...,1,bought this to go in my niece room you can tel...,
618,Extremely low in volume,0,extremely low in volume,
237,Our family really loves this new Echo,1,our family really loves this new echo,
2932,This device does not interact with my home fil...,0,this device does not interact with my home fil...,


In [29]:
test_set_sample=test_set.sample(20)
test_set_sample['pred_label']=''
test_set_sample

,review,label,clean_reviews,pred_label
376,Doesn't always respond when spoken to with pro...,0,doesn always respond when spoken to with promp...,
295,Easy to use and has great sound...It was a gre...,1,easy to use and has great sound it was great p...,
661,This Echo Dot is horrible. The volume on my ph...,0,this echo dot is horrible the volume on my pho...,
1487,Very disappointed. Alexa doesn’t hear me and ...,0,very disappointed alexa doesn hear me and does...,
2045,"The sound quality is not that good, especially...",0,the sound quality is not that good especially ...,
2251,"I don't have Alexa, but after getting this I'm...",1,don have alexa but after getting this feeling ...,
389,Almost like having Star Trek in your home! Fu...,1,almost like having star trek in your home fun ...,
1643,Love it!!,1,love it,
111,Sound is terrible if u want good music too get...,0,sound is terrible if want good music too get bose,
1340,Get rid of the &#34;Thongs To Try&#34; screen!...,0,get rid of the 34 thongs to try 34 screen numb...,


In [30]:
#Convert the dataframe to Json
json_data=test_set_sample[['clean_reviews','pred_label']].to_json(orient='records')
print(json_data)

[{"clean_reviews":"doesn always respond when spoken to with promp word thought it would connect to my echo upstairs and use it like an intercom haven been able to do that","pred_label":""},{"clean_reviews":"easy to use and has great sound it was great purchase at great price","pred_label":""},{"clean_reviews":"this echo dot is horrible the volume on my phone is louder than than this device my boss has one at this is not the case","pred_label":""},{"clean_reviews":"very disappointed alexa doesn hear me and doesn respond to my requests","pred_label":""},{"clean_reviews":"the sound quality is not that good especially for the price that you pay for this would recommend to get the echo dot and buy better speaker instead maybe sonos and use the echo dot","pred_label":""},{"clean_reviews":"don have alexa but after getting this feeling inspired to get one feel this is good gateway and or alexa trial and liking it so far channels work just fine like apple tv just need to check if there is phone

In [31]:
prompt = f"""
You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
{json_data}
```
"""

print(prompt)


You are an expert linguist, who is good at classifying customer review sentiments into Positive/Negative labels.
Help me classify customer reviews into: Positive(label=1), and Negative(label=0).
Customer reviews are provided between three back ticks.
In your output, only return the Json code back as output - which is provided between three backticks.
Your task is to update predicted labels under 'pred_label' in the Json code.
Don't make any changes to Json code format, please.

```
[{"clean_reviews":"doesn always respond when spoken to with promp word thought it would connect to my echo upstairs and use it like an intercom haven been able to do that","pred_label":""},{"clean_reviews":"easy to use and has great sound it was great purchase at great price","pred_label":""},{"clean_reviews":"this echo dot is horrible the volume on my phone is louder than than this device my boss has one at this is not the case","pred_label":""},{"clean_reviews":"very disappointed alexa doesn hear me and d

In [32]:
response=model.generate_content(prompt)
print(response.text)

```
[{"clean_reviews":"doesn always respond when spoken to with promp word thought it would connect to my echo upstairs and use it like an intercom haven been able to do that","pred_label":0},{"clean_reviews":"easy to use and has great sound it was great purchase at great price","pred_label":1},{"clean_reviews":"this echo dot is horrible the volume on my phone is louder than than this device my boss has one at this is not the case","pred_label":0},{"clean_reviews":"very disappointed alexa doesn hear me and doesn respond to my requests","pred_label":0},{"clean_reviews":"the sound quality is not that good especially for the price that you pay for this would recommend to get the echo dot and buy better speaker instead maybe sonos and use the echo dot","pred_label":1},{"clean_reviews":"don have alexa but after getting this feeling inspired to get one feel this is good gateway and or alexa trial and liking it so far channels work just fine like apple tv just need to check if there is phone 

In [33]:
import json

# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,clean_reviews,pred_label
0,doesn always respond when spoken to with promp...,0
1,easy to use and has great sound it was great p...,1
2,this echo dot is horrible the volume on my pho...,0
3,very disappointed alexa doesn hear me and does...,0
4,the sound quality is not that good especially ...,1
5,don have alexa but after getting this feeling ...,1
6,almost like having star trek in your home fun ...,1
7,love it,1
8,sound is terrible if want good music too get bose,0
9,get rid of the 34 thongs to try 34 screen numb...,0


In [34]:
test_set_sample['pred_label']=df_sample['pred_label'].values
test_set_sample

,review,label,clean_reviews,pred_label
376,Doesn't always respond when spoken to with pro...,0,doesn always respond when spoken to with promp...,0
295,Easy to use and has great sound...It was a gre...,1,easy to use and has great sound it was great p...,1
661,This Echo Dot is horrible. The volume on my ph...,0,this echo dot is horrible the volume on my pho...,0
1487,Very disappointed. Alexa doesn’t hear me and ...,0,very disappointed alexa doesn hear me and does...,0
2045,"The sound quality is not that good, especially...",0,the sound quality is not that good especially ...,1
2251,"I don't have Alexa, but after getting this I'm...",1,don have alexa but after getting this feeling ...,1
389,Almost like having Star Trek in your home! Fu...,1,almost like having star trek in your home fun ...,1
1643,Love it!!,1,love it,1
111,Sound is terrible if u want good music too get...,0,sound is terrible if want good music too get bose,0
1340,Get rid of the &#34;Thongs To Try&#34; screen!...,0,get rid of the 34 thongs to try 34 screen numb...,0


In [35]:
#plotting the confusion matrix on the predictions 
from sklearn.metrics import accuracy_score,confusion_matrix

y_true=test_set_sample["label"]
y_pred=test_set_sample["pred_label"]

confusion_matrix(y_true,y_pred)

array([[ 8,  1],
       [ 0, 11]])

In [36]:
print("Accuracy Score=",accuracy_score(y_true,y_pred))

Accuracy Score= 0.95
